# H&M Data Sampling and Ingest

## Setup

In [22]:
%%capture
%pip install python-dotenv graphdatascience neo4j_tools

In [23]:
import pandas as pd

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [24]:
from dotenv import load_dotenv
import os
load_dotenv('../.streamlit/secrets.toml', override=True)

# Neo4j
NEO4J_URI = os.getenv('HM_NEO4J_URI')
NEO4J_USERNAME = os.getenv('HM_NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('HM_NEO4J_PASSWORD')
AURA_DS = eval(os.getenv('HM_AURA_DS').title())

#OPENAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## Knowledge Graph Building

<img src="img/hm-banner.png" alt="summary" width="2000"/>

We will build our knowledge graph from the [H&M Personalized Fashion Recommendations Dataset](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data), a sample of real customer purchase data that includes rich information around products including names, types, descriptions, department sections, etc.

Below is the graph data model we will use:

<img src="img/hm-data-model.png" alt="summary" width="1000"/>


### Connect to Neo4j

We will use the [Graph Data Science Python Client](https://neo4j.com/docs/graph-data-science-client/current/) to connect to Neo4j. This client will allow us to easily run [Graph Data Science](https://neo4j.com/docs/graph-data-science/current/introduction/) algorithms from Python.

This client will only work if your Neo4j instance has Graph Data Science installed.  If not, you can still use the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/current/) or use Langchain’s Neo4j Graph object.

In [25]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=AURA_DS)

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

### Get Data, Create Constraints, and Load

Before loading data into Neo4j, it is usually best practice to create Key or Uniqueness constraints for nodes. These [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) act as an index with some validation on unique id properties and thus make `MATCH` statements run significantly faster. Not doing this can result in a VERY slow ingest, so this is a critical step.

We will be using convenience functions for loading nodes and relationships in batch. As the data load runs, you can see the Cypher statements being used.

In [26]:
%%time

import pandas as pd
from neo4j_tools import gds_db_load

# get source data - it has been pre-formatted. If you would like to re-generate from source on Kaggle, see the data-prep.ipynb notebook
department_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/department.csv')
product_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/product.csv')
product_df.detailDesc = product_df.detailDesc.fillna('')
article_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/article.csv')
customer_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/customer.csv')
transaction_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/transaction.csv')

# create constraints - one uniqueness constraint for each node label
gds.run_cypher('CREATE CONSTRAINT unique_department_no IF NOT EXISTS FOR (n:Department) REQUIRE n.departmentNo IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_product_code IF NOT EXISTS FOR (n:Product) REQUIRE n.productCode IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_article_id IF NOT EXISTS FOR (n:Article) REQUIRE n.articleId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_customer_id IF NOT EXISTS FOR (n:Customer) REQUIRE n.customerId IS UNIQUE')

# load nodes
gds_db_load.load_nodes(gds, department_df, 'departmentNo', 'Department')
gds_db_load.load_nodes(gds, article_df.drop(columns=['productCode', 'departmentNo']), 'articleId', 'Article')
gds_db_load.load_nodes(gds, product_df, 'productCode', 'Product')
gds_db_load.load_nodes(gds, customer_df, 'customerId', 'Customer')

# load relationships
gds_db_load.load_rels(gds, article_df[['articleId', 'departmentNo']], source_target_labels=('Article', 'Department'),
                      source_node_key='articleId', target_node_key='departmentNo',
                      rel_type='FROM_DEPARTMENT')
gds_db_load.load_rels(gds, article_df[['articleId', 'productCode']], source_target_labels=('Article', 'Product'),
                      source_node_key='articleId',target_node_key='productCode',
                      rel_type='VARIANT_OF')
gds_db_load.load_rels(gds, transaction_df, source_target_labels=('Customer', 'Article'),
                      source_node_key='customerId', target_node_key='articleId', rel_key='txId',
                      rel_type='PURCHASED')

# convert transaction dates
gds.run_cypher('''
MATCH (:Customer)-[r:PURCHASED]->()
SET r.tDat = date(r.tDat)
''')

# create combined text property. This will help simplify later with semantic search and RAG
gds.run_cypher("""
    MATCH(p:Product)
    SET p.text = '##Product\n' +
        'Name: ' + p.prodName + '\n' +
        'Type: ' + p.productTypeName + '\n' +
        'Group: ' + p.productGroupName + '\n' +
        'Garment Type: ' + p.garmentGroupName + '\n' +
        'Description: ' + p.detailDesc
    RETURN count(p) AS propertySetCount
    """)

======  loading Department nodes  ======
staging 266 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Department {departmentNo: rec.departmentNo})
SET n.departmentName = rec.departmentName, n.sectionNo = rec.sectionNo, n.sectionName = rec.sectionName
RETURN count(n) AS nodeLoadedCount
```

Loaded 266 of 266 nodes
======  loading Article nodes  ======
staging 13,351 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Article {articleId: rec.articleId})
SET n.prodName = rec.prodName, n.productTypeName = rec.productTypeName, n.graphicalAppearanceNo = rec.graphicalAppearanceNo, n.graphicalAppearanceName = rec.graphicalAppearanceName, n.colourGroupCode = rec.colourGroupCode, n.colourGroupName = rec.colourGroupName
RETURN count(n) AS nodeLoadedCount
```

Loaded 10,000 of 13,351 nodes
Loaded 13,351 of 13,351 nodes
======  loading Product nodes  ======
staging 8,044 records

Using This Cypher Query:
```
UNWIND $recs AS rec
MERGE(n:Product {productCode: rec.produc

,propertySetCount
0,8044


## Create Embeddings and Vector Index

In this section, we generate text embeddings out of product descriptions and create a vector index to power search and retrieval.


In [28]:
embedding_dimension = 1536 #default for OpenAI text-embedding-ada-002

#generate embeddings
gds.run_cypher('''
MATCH (n:Product) WHERE size(n.detailDesc) <> 0
WITH collect(n) AS nodes, toInteger(rand()*$numberOfBatches) AS partition
CALL {
    WITH nodes
    CALL genai.vector.encodeBatch([node IN nodes| node.text], "OpenAI", { token: $token})
    YIELD index, vector
    CALL db.create.setNodeVectorProperty(nodes[index], "textEmbedding", vector)
} IN TRANSACTIONS OF 1 ROW''', params={'token':OPENAI_API_KEY, 'numberOfBatches':80})

#create vector index
gds.run_cypher('''
CREATE VECTOR INDEX product_text_embeddings IF NOT EXISTS FOR (n:Product) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dimension),
 `vector.similarity_function`: 'cosine'
}}''', params={'dimension': embedding_dimension})

#wait for index to come online
gds.run_cypher('CALL db.awaitIndex("product_text_embeddings", 300)')

""


Test vector search

In [29]:
search_prompt = 'Oversized Sweaters'

gds.run_cypher('''
WITH genai.vector.encode($searchPrompt, "OpenAI", { token: $token}) AS queryVector
CALL db.index.vector.queryNodes("product_text_embeddings", 10, queryVector)
YIELD node AS product, score
RETURN product.productCode AS productCode,
    product.text AS text,
    score
''', params={'searchPrompt': search_prompt, 'token':OPENAI_API_KEY})

,productCode,text,score
0,842001,"##Product\nName: Betsy Oversized\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Oversized, V-neck jumper in a soft, loose knit containing some wool and alpaca wool. Dropped shoulders, long, wide sleeves, wide ribbing around the neckline, cuffs and hem, and slits in the sides.",0.947626
1,817392,"##Product\nName: Japp oversize sweater\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Jersey Basic\nDescription: Relaxed-fit top in sweatshirt fabric with a ribbed turtle neck, dropped shoulders, long, wide sleeves and ribbing at the cuffs and hem. Longer at the back.",0.946449
2,709418,"##Product\nName: DIV Anni oversize hood\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Unknown\nDescription: Oversized top in sweatshirt fabric with a lined hood with a wrapover front. Kangaroo pocket, dropped shoulders, long sleeves and ribbing at the cuffs and hem. Soft brushed inside.",0.933464
3,860833,"##Product\nName: Runar sweater\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Jersey Basic\nDescription: Oversized top in soft sweatshirt fabric. Relaxed fit with low dropped shoulders, extra-long sleeves and ribbing around the neckline, cuffs and hem. Soft brushed inside.",0.932758
4,893141,"##Product\nName: Sandy\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Oversized jumper in a soft knit containing some wool with dropped shoulders, long sleeves, a rounded hem and ribbing around the neckline, cuffs and hem. Longer at the back. The polyester content of the jumper is recycled.",0.931518
5,690624,"##Product\nName: Happy\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Oversized jumper in a soft cable knit with a polo neck, low dropped shoulders and long, wide sleeves. Straight hem and ribbing at the cuffs and hem. Slightly longer at the back.",0.931496
6,594834,"##Product\nName: Dolly hood\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Jersey Fancy\nDescription: Oversized top in soft sweatshirt fabric with a lined drawstring hood, dropped shoulders and ribbing at the cuffs and hem.",0.930855
7,620083,"##Product\nName: All in sweater\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Jersey Fancy\nDescription: Cropped top in sweatshirt fabric with long sleeves and ribbing around the neckline, cuffs and hem.",0.930159
8,690623,"##Product\nName: Simba\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Oversized jumper in a soft knit containing some wool with a cowl neck, long raglan sleeves and wide ribbing at the cuffs and hem.",0.929556
9,812167,"##Product\nName: Macy\nType: Sweater\nGroup: Garment Upper body\nGarment Type: Knitwear\nDescription: Oversized jumper in a soft knit containing some wool with a ribbed polo neck, low dropped shoulders, long sleeves, and ribbing at the cuffs and hem. The polyester content of the jumper is recycled.",0.929450
